## Titanic Project 
####              Ahmet Salim Çetin

# First we import and try to understand the data


In [1]:
# import required libraries
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from sklearn.linear_model import LinearRegression 
LR = LinearRegression()
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score,mean_absolute_error
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB

import warnings
warnings.filterwarnings('ignore')

In [2]:
ttrain = pd.read_csv("ttrain.csv")
ttest = pd.read_csv("ttest.csv")

In [3]:
titan=ttrain.append(ttest)

In [4]:
titan = titan.reset_index(drop=True)

In [5]:
titan.shape

(1309, 12)

In [6]:
titan.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  1309 non-null   int64  
 1   Survived     891 non-null    float64
 2   Pclass       1309 non-null   int64  
 3   Name         1309 non-null   object 
 4   Sex          1309 non-null   object 
 5   Age          1046 non-null   float64
 6   SibSp        1309 non-null   int64  
 7   Parch        1309 non-null   int64  
 8   Ticket       1309 non-null   object 
 9   Fare         1308 non-null   float64
 10  Cabin        295 non-null    object 
 11  Embarked     1307 non-null   object 
dtypes: float64(3), int64(4), object(5)
memory usage: 122.8+ KB


In [7]:
titan.shape

(1309, 12)

In [8]:
titan.isnull().sum()

PassengerId       0
Survived        418
Pclass            0
Name              0
Sex               0
Age             263
SibSp             0
Parch             0
Ticket            0
Fare              1
Cabin          1014
Embarked          2
dtype: int64

In [9]:
titan.describe(include="all")

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
count,1309.000000,891.000000,1309.000000,1309,1309,1046.000000,1309.000000,1309.000000,1309,1308.000000,295,1307
unique,NaN,NaN,NaN,1307,2,NaN,NaN,NaN,929,NaN,186,3
top,NaN,NaN,NaN,"Connolly, Miss. Kate",male,NaN,NaN,NaN,CA. 2343,NaN,C23 C25 C27,S
freq,NaN,NaN,NaN,2,843,NaN,NaN,NaN,11,NaN,6,914
mean,655.000000,0.383838,2.294882,NaN,NaN,29.881138,0.498854,0.385027,NaN,33.295479,NaN,NaN
std,378.020061,0.486592,0.837836,NaN,NaN,14.413493,1.041658,0.865560,NaN,51.758668,NaN,NaN
min,1.000000,0.000000,1.000000,NaN,NaN,0.170000,0.000000,0.000000,NaN,0.000000,NaN,NaN
25%,328.000000,0.000000,2.000000,NaN,NaN,21.000000,0.000000,0.000000,NaN,7.895800,NaN,NaN
50%,655.000000,0.000000,3.000000,NaN,NaN,28.000000,0.000000,0.000000,NaN,14.454200,NaN,NaN
75%,982.000000,1.000000,3.000000,NaN,NaN,39.000000,1.000000,0.000000,NaN,31.275000,NaN,NaN


In [10]:
titan.corr()['Survived'].sort_values(ascending=False)


Survived       1.000000
Fare           0.257307
Parch          0.081629
PassengerId   -0.005007
SibSp         -0.035322
Age           -0.077221
Pclass        -0.338481
Name: Survived, dtype: float64

now it is time to do some feature engineering

In [11]:
titan = titan.drop(['Cabin'], axis = 1)

In [12]:
titan = titan.drop(['Ticket'], axis = 1)

In [13]:
titan.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Embarked
0,1,0.0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,7.2500,S
1,2,1.0,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,71.2833,C
2,3,1.0,3,"Heikkinen, Miss. Laina",female,26.0,0,0,7.9250,S
3,4,1.0,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,53.1000,S
4,5,0.0,3,"Allen, Mr. William Henry",male,35.0,0,0,8.0500,S


In [14]:
titan['Title'] = titan['Name'].str.extract(' ([A-Za-z]+)\.', expand = False)
titan['Title'].value_counts()

Mr          757
Miss        260
Mrs         197
Master       61
Rev           8
Dr            8
Col           4
Ms            2
Major         2
Mlle          2
Capt          1
Lady          1
Dona          1
Jonkheer      1
Don           1
Mme           1
Sir           1
Countess      1
Name: Title, dtype: int64

In [15]:
titan['Title'] = titan['Title'].replace(["Ms","Mlle"],"Miss")
titan['Title'] = titan['Title'].replace(["Mme","Countess","Lady","Dona"],"Mrs")
titan['Title'] = titan['Title'].replace(["Dr","Major","Col","Sir","Rev","Jonkheer","Don","Capt"],"Mr")

In [16]:
titan["Age"].fillna(titan.groupby("Title")["Age"].transform("median"),inplace=True)

In [17]:
titan.isnull().sum()

PassengerId      0
Survived       418
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Fare             1
Embarked         2
Title            0
dtype: int64

In [18]:
titan["Fare"].fillna(titan["Fare"].mean(), inplace =True)


In [19]:
titan["FamilySize"] = titan["SibSp"] + titan["Parch"] + 1


In [20]:
titan.describe(include='all')

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Embarked,Title,FamilySize
count,1309.000000,891.000000,1309.000000,1309,1309,1309.000000,1309.000000,1309.000000,1309.000000,1307,1309,1309.000000
unique,NaN,NaN,NaN,1307,2,NaN,NaN,NaN,NaN,3,4,NaN
top,NaN,NaN,NaN,"Connolly, Miss. Kate",male,NaN,NaN,NaN,NaN,S,Mr,NaN
freq,NaN,NaN,NaN,2,843,NaN,NaN,NaN,NaN,914,783,NaN
mean,655.000000,0.383838,2.294882,NaN,NaN,29.547876,0.498854,0.385027,33.295479,NaN,NaN,1.883881
std,378.020061,0.486592,0.837836,NaN,NaN,13.154383,1.041658,0.865560,51.738879,NaN,NaN,1.583639
min,1.000000,0.000000,1.000000,NaN,NaN,0.170000,0.000000,0.000000,0.000000,NaN,NaN,1.000000
25%,328.000000,0.000000,2.000000,NaN,NaN,22.000000,0.000000,0.000000,7.895800,NaN,NaN,1.000000
50%,655.000000,0.000000,3.000000,NaN,NaN,30.000000,0.000000,0.000000,14.454200,NaN,NaN,1.000000
75%,982.000000,1.000000,3.000000,NaN,NaN,35.500000,1.000000,0.000000,31.275000,NaN,NaN,2.000000


In [21]:
titan.isnull().sum()

PassengerId      0
Survived       418
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Fare             0
Embarked         2
Title            0
FamilySize       0
dtype: int64

In [22]:
titan['Embarked'].value_counts() 


S    914
C    270
Q    123
Name: Embarked, dtype: int64

In [23]:
titan = titan.fillna({"Embarked": "S"})

In [24]:
titan = titan.drop(['SibSp'], axis = 1)


In [25]:
titan = titan.drop(['Parch'], axis = 1)


In [26]:
titan.describe(include='all')

,PassengerId,Survived,Pclass,Name,Sex,Age,Fare,Embarked,Title,FamilySize
count,1309.000000,891.000000,1309.000000,1309,1309,1309.000000,1309.000000,1309,1309,1309.000000
unique,NaN,NaN,NaN,1307,2,NaN,NaN,3,4,NaN
top,NaN,NaN,NaN,"Connolly, Miss. Kate",male,NaN,NaN,S,Mr,NaN
freq,NaN,NaN,NaN,2,843,NaN,NaN,916,783,NaN
mean,655.000000,0.383838,2.294882,NaN,NaN,29.547876,33.295479,NaN,NaN,1.883881
std,378.020061,0.486592,0.837836,NaN,NaN,13.154383,51.738879,NaN,NaN,1.583639
min,1.000000,0.000000,1.000000,NaN,NaN,0.170000,0.000000,NaN,NaN,1.000000
25%,328.000000,0.000000,2.000000,NaN,NaN,22.000000,7.895800,NaN,NaN,1.000000
50%,655.000000,0.000000,3.000000,NaN,NaN,30.000000,14.454200,NaN,NaN,1.000000
75%,982.000000,1.000000,3.000000,NaN,NaN,35.500000,31.275000,NaN,NaN,2.000000


In [27]:
titan.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,Fare,Embarked,Title,FamilySize
0,1,0.0,3,"Braund, Mr. Owen Harris",male,22.0,7.2500,S,Mr,2
1,2,1.0,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,71.2833,C,Mrs,2
2,3,1.0,3,"Heikkinen, Miss. Laina",female,26.0,7.9250,S,Miss,1
3,4,1.0,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,53.1000,S,Mrs,2
4,5,0.0,3,"Allen, Mr. William Henry",male,35.0,8.0500,S,Mr,1


In [28]:
titan = titan.drop(['Name'], axis = 1)


In [29]:
titanic=pd.get_dummies(titan,drop_first=True)


In [30]:
titanic.describe(include='all')

,PassengerId,Survived,Pclass,Age,Fare,FamilySize,Sex_male,Embarked_Q,Embarked_S,Title_Miss,Title_Mr,Title_Mrs
count,1309.000000,891.000000,1309.000000,1309.000000,1309.000000,1309.000000,1309.000000,1309.000000,1309.000000,1309.000000,1309.000000,1309.000000
mean,655.000000,0.383838,2.294882,29.547876,33.295479,1.883881,0.644003,0.093965,0.699771,0.201681,0.598167,0.153552
std,378.020061,0.486592,0.837836,13.154383,51.738879,1.583639,0.478997,0.291891,0.458533,0.401408,0.490456,0.360657
min,1.000000,0.000000,1.000000,0.170000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,328.000000,0.000000,2.000000,22.000000,7.895800,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,655.000000,0.000000,3.000000,30.000000,14.454200,1.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000
75%,982.000000,1.000000,3.000000,35.500000,31.275000,2.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000
max,1309.000000,1.000000,3.000000,80.000000,512.329200,11.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [31]:
titanic

,PassengerId,Survived,Pclass,Age,Fare,FamilySize,Sex_male,Embarked_Q,Embarked_S,Title_Miss,Title_Mr,Title_Mrs
0,1,0.0,3,22.0,7.2500,2,1,0,1,0,1,0
1,2,1.0,1,38.0,71.2833,2,0,0,0,0,0,1
2,3,1.0,3,26.0,7.9250,1,0,0,1,1,0,0
3,4,1.0,1,35.0,53.1000,2,0,0,1,0,0,1
4,5,0.0,3,35.0,8.0500,1,1,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...
1304,1305,NaN,3,30.0,8.0500,1,1,0,1,0,1,0
1305,1306,NaN,1,39.0,108.9000,1,0,0,0,0,0,1
1306,1307,NaN,3,38.5,7.2500,1,1,0,1,0,1,0
1307,1308,NaN,3,30.0,8.0500,1,1,0,1,0,1,0


In [32]:
ttrain.tail(20)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
871,872,1,1,"Beckwith, Mrs. Richard Leonard (Sallie Monypeny)",female,47.0,1,1,11751,52.5542,D35,S
872,873,0,1,"Carlsson, Mr. Frans Olof",male,33.0,0,0,695,5.0000,B51 B53 B55,S
873,874,0,3,"Vander Cruyssen, Mr. Victor",male,47.0,0,0,345765,9.0000,NaN,S
874,875,1,2,"Abelson, Mrs. Samuel (Hannah Wizosky)",female,28.0,1,0,P/PP 3381,24.0000,NaN,C
875,876,1,3,"Najib, Miss. Adele Kiamie ""Jane""",female,15.0,0,0,2667,7.2250,NaN,C
876,877,0,3,"Gustafsson, Mr. Alfred Ossian",male,20.0,0,0,7534,9.8458,NaN,S
877,878,0,3,"Petroff, Mr. Nedelio",male,19.0,0,0,349212,7.8958,NaN,S
878,879,0,3,"Laleff, Mr. Kristo",male,NaN,0,0,349217,7.8958,NaN,S
879,880,1,1,"Potter, Mrs. Thomas Jr (Lily Alexenia Wilson)",female,56.0,0,1,11767,83.1583,C50,C
880,881,1,2,"Shelley, Mrs. William (Imanita Parrish Hall)",female,25.0,0,1,230433,26.0000,NaN,S


In [33]:
titanic.head()

,PassengerId,Survived,Pclass,Age,Fare,FamilySize,Sex_male,Embarked_Q,Embarked_S,Title_Miss,Title_Mr,Title_Mrs
0,1,0.0,3,22.0,7.2500,2,1,0,1,0,1,0
1,2,1.0,1,38.0,71.2833,2,0,0,0,0,0,1
2,3,1.0,3,26.0,7.9250,1,0,0,1,1,0,0
3,4,1.0,1,35.0,53.1000,2,0,0,1,0,0,1
4,5,0.0,3,35.0,8.0500,1,1,0,1,0,1,0


In [34]:
titanic.shape

(1309, 12)

In [35]:
titanic[880:900]

,PassengerId,Survived,Pclass,Age,Fare,FamilySize,Sex_male,Embarked_Q,Embarked_S,Title_Miss,Title_Mr,Title_Mrs
880,881,1.0,2,25.0,26.0000,2,0,0,1,0,0,1
881,882,0.0,3,33.0,7.8958,1,1,0,1,0,1,0
882,883,0.0,3,22.0,10.5167,1,0,0,1,1,0,0
883,884,0.0,2,28.0,10.5000,1,1,0,1,0,1,0
884,885,0.0,3,25.0,7.0500,1,1,0,1,0,1,0
885,886,0.0,3,39.0,29.1250,6,0,1,0,0,0,1
886,887,0.0,2,27.0,13.0000,1,1,0,1,0,1,0
887,888,1.0,1,19.0,30.0000,1,0,0,1,1,0,0
888,889,0.0,3,22.0,23.4500,4,0,0,1,1,0,0
889,890,1.0,1,26.0,30.0000,1,1,0,0,0,1,0


In [36]:
traindata = titanic[:891]
testdata = titanic[891:]



In [37]:
x=traindata.drop(['Survived', 'PassengerId'], axis=1)


In [38]:
y=traindata['Survived']

In [39]:
x.shape

(891, 10)

In [40]:
x

,Pclass,Age,Fare,FamilySize,Sex_male,Embarked_Q,Embarked_S,Title_Miss,Title_Mr,Title_Mrs
0,3,22.0,7.2500,2,1,0,1,0,1,0
1,1,38.0,71.2833,2,0,0,0,0,0,1
2,3,26.0,7.9250,1,0,0,1,1,0,0
3,1,35.0,53.1000,2,0,0,1,0,0,1
4,3,35.0,8.0500,1,1,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...
886,2,27.0,13.0000,1,1,0,1,0,1,0
887,1,19.0,30.0000,1,0,0,1,1,0,0
888,3,22.0,23.4500,4,0,0,1,1,0,0
889,1,26.0,30.0000,1,1,0,0,0,1,0


In [41]:
y.shape

(891,)

In [42]:
y

0      0.0
1      1.0
2      1.0
3      1.0
4      0.0
      ... 
886    0.0
887    1.0
888    0.0
889    1.0
890    0.0
Name: Survived, Length: 891, dtype: float64

In [43]:
x_train, x_test,y_train, y_test = train_test_split(x,y,test_size=.2,random_state=100)


In [44]:
x_train.shape, x_test.shape,y_train.shape, y_test.shape 


((712, 10), (179, 10), (712,), (179,))

now we will try different methods and look for the most accurate one

In [45]:
# gaussian
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score

gaussian = GaussianNB()
gaussian.fit(x_train, y_train)
y_pred = gaussian.predict(x_test)
acc_gaussian = round(accuracy_score(y_pred, y_test) * 100, 2)
print(acc_gaussian)

78.77


In [46]:
# Logistic Regression
from sklearn.linear_model import LogisticRegression

logreg = LogisticRegression()
logreg.fit(x_train, y_train)
y_pred = logreg.predict(x_test)
acc_logreg = round(accuracy_score(y_pred, y_test) * 100, 2)
print(acc_logreg)

82.68


In [47]:
# Decision Tree
from sklearn.tree import DecisionTreeClassifier

decisiontree = DecisionTreeClassifier()
decisiontree.fit(x_train, y_train)
y_pred = decisiontree.predict(x_test)
acc_decisiontree = round(accuracy_score(y_pred, y_test) * 100, 2)
print(acc_decisiontree)

77.65


In [48]:
# Random Forest
from sklearn.ensemble import RandomForestClassifier

randomforest = RandomForestClassifier()
randomforest.fit(x_train, y_train)
y_pred = randomforest.predict(x_test)
acc_randomforest = round(accuracy_score(y_pred, y_test) * 100, 2)
print(acc_randomforest)

81.01


In [49]:
# k-Nearest Neighbors
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier()
knn.fit(x_train, y_train)
y_pred = knn.predict(x_test)
acc_knn = round(accuracy_score(y_pred, y_test) * 100, 2)
print(acc_knn)

70.39


In [50]:
# Gradient Boosting Classifier
from sklearn.ensemble import GradientBoostingClassifier

gbk = GradientBoostingClassifier()
gbk.fit(x_train, y_train)
y_pred = gbk.predict(x_test)
acc_gbk = round(accuracy_score(y_pred, y_test) * 100, 2)
print(acc_gbk)


84.36


it seems that gradient boosting is valid

In [51]:
x1=testdata.drop(['Survived', 'PassengerId'], axis=1)

In [52]:
x1

,Pclass,Age,Fare,FamilySize,Sex_male,Embarked_Q,Embarked_S,Title_Miss,Title_Mr,Title_Mrs
891,3,34.5,7.8292,1,1,1,0,0,1,0
892,3,47.0,7.0000,2,0,0,1,0,0,1
893,2,62.0,9.6875,1,1,1,0,0,1,0
894,3,27.0,8.6625,1,1,0,1,0,1,0
895,3,22.0,12.2875,3,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...
1304,3,30.0,8.0500,1,1,0,1,0,1,0
1305,1,39.0,108.9000,1,0,0,0,0,0,1
1306,3,38.5,7.2500,1,1,0,1,0,1,0
1307,3,30.0,8.0500,1,1,0,1,0,1,0


In [53]:
y1=testdata['Survived']

In [54]:
y1

891    NaN
892    NaN
893    NaN
894    NaN
895    NaN
        ..
1304   NaN
1305   NaN
1306   NaN
1307   NaN
1308   NaN
Name: Survived, Length: 418, dtype: float64

In [55]:
ids = testdata['PassengerId']


In [56]:
ids

891      892
892      893
893      894
894      895
895      896
        ... 
1304    1305
1305    1306
1306    1307
1307    1308
1308    1309
Name: PassengerId, Length: 418, dtype: int64

In [57]:
testdata['Survived']=gbk.predict(x1)

In [58]:
testdata.head()

,PassengerId,Survived,Pclass,Age,Fare,FamilySize,Sex_male,Embarked_Q,Embarked_S,Title_Miss,Title_Mr,Title_Mrs
891,892,0.0,3,34.5,7.8292,1,1,1,0,0,1,0
892,893,0.0,3,47.0,7.0000,2,0,0,1,0,0,1
893,894,0.0,2,62.0,9.6875,1,1,1,0,0,1,0
894,895,0.0,3,27.0,8.6625,1,1,0,1,0,1,0
895,896,1.0,3,22.0,12.2875,3,0,0,1,0,0,1


In [59]:
titanic2=testdata.append(traindata)

In [60]:
titanic2.isnull().sum()

PassengerId    0
Survived       0
Pclass         0
Age            0
Fare           0
FamilySize     0
Sex_male       0
Embarked_Q     0
Embarked_S     0
Title_Miss     0
Title_Mr       0
Title_Mrs      0
dtype: int64

In [61]:
titanic2.tail(20)

,PassengerId,Survived,Pclass,Age,Fare,FamilySize,Sex_male,Embarked_Q,Embarked_S,Title_Miss,Title_Mr,Title_Mrs
871,872,1.0,1,47.0,52.5542,3,0,0,1,0,0,1
872,873,0.0,1,33.0,5.0000,1,1,0,1,0,1,0
873,874,0.0,3,47.0,9.0000,1,1,0,1,0,1,0
874,875,1.0,2,28.0,24.0000,2,0,0,0,0,0,1
875,876,1.0,3,15.0,7.2250,1,0,0,0,1,0,0
876,877,0.0,3,20.0,9.8458,1,1,0,1,0,1,0
877,878,0.0,3,19.0,7.8958,1,1,0,1,0,1,0
878,879,0.0,3,30.0,7.8958,1,1,0,1,0,1,0
879,880,1.0,1,56.0,83.1583,2,0,0,0,0,0,1
880,881,1.0,2,25.0,26.0000,2,0,0,1,0,0,1


as we see we filled blanks in the test data with predicted values by using machine learning from train data

In [62]:
testdata['Survived']

891     0.0
892     0.0
893     0.0
894     0.0
895     1.0
       ... 
1304    0.0
1305    1.0
1306    0.0
1307    0.0
1308    1.0
Name: Survived, Length: 418, dtype: float64

In [63]:
testdata['Survivedw'] = testdata['Survived'].astype(int)

In [66]:
testdata

,PassengerId,Survived,Pclass,Age,Fare,FamilySize,Sex_male,Embarked_Q,Embarked_S,Title_Miss,Title_Mr,Title_Mrs,Survivedw
891,892,0.0,3,34.5,7.8292,1,1,1,0,0,1,0,0
892,893,0.0,3,47.0,7.0000,2,0,0,1,0,0,1,0
893,894,0.0,2,62.0,9.6875,1,1,1,0,0,1,0,0
894,895,0.0,3,27.0,8.6625,1,1,0,1,0,1,0,0
895,896,1.0,3,22.0,12.2875,3,0,0,1,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1304,1305,0.0,3,30.0,8.0500,1,1,0,1,0,1,0,0
1305,1306,1.0,1,39.0,108.9000,1,0,0,0,0,0,1,1
1306,1307,0.0,3,38.5,7.2500,1,1,0,1,0,1,0,0
1307,1308,0.0,3,30.0,8.0500,1,1,0,1,0,1,0,0


In [67]:
testdata.drop('Survived', inplace=True, axis=1)


In [68]:
testdata['Survived'] = testdata['Survivedw'].astype(int)


In [69]:
testdata.drop('Survivedw', inplace=True, axis=1)


In [70]:
testdata

,PassengerId,Pclass,Age,Fare,FamilySize,Sex_male,Embarked_Q,Embarked_S,Title_Miss,Title_Mr,Title_Mrs,Survived
891,892,3,34.5,7.8292,1,1,1,0,0,1,0,0
892,893,3,47.0,7.0000,2,0,0,1,0,0,1,0
893,894,2,62.0,9.6875,1,1,1,0,0,1,0,0
894,895,3,27.0,8.6625,1,1,0,1,0,1,0,0
895,896,3,22.0,12.2875,3,0,0,1,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...
1304,1305,3,30.0,8.0500,1,1,0,1,0,1,0,0
1305,1306,1,39.0,108.9000,1,0,0,0,0,0,1,1
1306,1307,3,38.5,7.2500,1,1,0,1,0,1,0,0
1307,1308,3,30.0,8.0500,1,1,0,1,0,1,0,0


In [71]:
output = pd.DataFrame({ 'PassengerId' : ids, 'Survived': testdata['Survived'] })
output.to_csv('submission5.csv', index=False)